In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import time
from torch.optim import AdamW

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

print(device)

cuda


In [ ]:
# Encontra diretorio atual
atual_dir = os.getcwd()

parent_dir = os.path.split(atual_dir)

parent_dir = os.path.split(parent_dir[0])

parent_dir = os.path.split(parent_dir[0])

caminho_csv = os.path.join(parent_dir[0], "Pre-processamento\\noticias_dados_limpos.csv")

df = pd.read_csv(caminho_csv)

df.head()

,Texto,Categoria
0,O Podemos decidiu expulsar o deputado federal ...,Real
1,"Bolsonaro é um liberal completo, diz president...",Real
2,Ministro do STF libera Andrea Neves de prisão ...,Real
3,"Apesar da abundância, cresce preocupação com p...",Real
4,"Por que Harvard e MIT levarão Dilma, Moro e Su...",Real


In [4]:
# Aplicação do LabelEncoder para transformar a variável categórica 'Categoria' em uma variável numérica 'label'.

le = LabelEncoder()

df['label'] = le.fit_transform(df['Categoria'])

df['Categoria'].unique(), df['label'].unique()

(array(['Real', 'Falso'], dtype=object), array([1, 0]))

In [5]:
# Divide os dados em um conjunto de treino (70%) e um conjunto temporário (30%)
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)

# Divide o conjunto temporário em conjuntos de validação (15%) e teste (15%)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [6]:
# BERT tokenizer
bert_tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased",
                                               do_lower_case=False)

c:\Users\mathe\anaconda3\envs\detector\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        # Inicializa o dataset com textos, rótulos, tokenizer e o comprimento máximo da sequência
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        # Retorna o número de amostras no dataset
        return len(self.texts)

    def __getitem__(self, idx):
        # Converte o texto na posição 'idx' para string
        text = str(self.texts[idx])
        label = self.labels[idx]

        # Tokeniza o texto, aplicando truncamento, padding, e retornando tensores PyTorch
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True, # Adiciona tokens especiais [CLS] e [SEP]
            max_length=self.max_len, # Define o comprimento máximo da sequência
            return_token_type_ids=False, # Não retorna ids de tipos de token (ex.: segmentação de sentenças)
            padding='max_length', # Aplica padding até o comprimento máximo
            truncation=True, # Trunca sequências maiores que o comprimento máximo
            return_attention_mask=True, # Retorna a máscara de atenção
            return_tensors='pt', # Retorna os dados como tensores PyTorch
        )

        # Retorna um dicionário com os tensores 'input_ids', 'attention_mask' e 'labels'
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [8]:
def create_data_loader(df, tokenizer, max_len, batch_size):
    # Cria um dataset usando a classe NewsDataset
    ds = NewsDataset(
        texts=df.Texto.to_numpy(),
        labels=df.label.to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
    )

    # Retorna um DataLoader para o dataset, dividindo-o em batches
    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=0
    )

BATCH_SIZE = 16
MAX_LEN = 128 # Define o comprimento máximo das sequências

# Criação dos DataLoaders para treino, validação e teste
train_data_loader = create_data_loader(train_df, bert_tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(val_df, bert_tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(test_df, bert_tokenizer, MAX_LEN, BATCH_SIZE)


In [9]:
# Carrega o modelo BERT pré-treinado com a cabeça de classificação de sequência
model = BertForSequenceClassification.from_pretrained(
    "neuralmind/bert-base-portuguese-cased",
    num_labels=2, # Define o número de classes (Real, Fake)
    output_attentions=False,
    output_hidden_states=False
)

model = model.to(device) # Move o modelo para o dispositivo (CPU/GPU)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Configura o otimizador AdamW com uma pequena taxa de aprendizado
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define o número total de passos de treinamento e cria um agendador de aprendizado
total_steps = len(train_data_loader) * 3  

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

# Define a função de perda como CrossEntropyLoss
loss_fn = torch.nn.CrossEntropyLoss().to(device)


In [11]:
def train_epoch(
    model,
    data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    n_examples
):
    model = model.train() # Coloca o modelo em modo de treinamento

    losses = []
    correct_predictions = 0

    for d in tqdm(data_loader, desc="Training"): # Loop sobre batches de dados
        input_ids = d["input_ids"].to(device) # Move input_ids para o dispositivo
        attention_mask = d["attention_mask"].to(device) # Move attention_mask para o dispositivo
        labels = d["labels"].to(device) # Move labels para o dispositivo

        # Passa os dados pelo modelo
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        loss = loss_fn(outputs.logits, labels) # Calcula a perda
        correct_predictions += torch.sum(torch.argmax(outputs.logits, dim=1) == labels) # Conta predições corretas
        losses.append(loss.item()) # Armazena a perda para análise

        loss.backward() # Calcula gradientes
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # Clipa os gradientes para evitar explosões
        optimizer.step() # Atualiza os pesos do modelo
        scheduler.step() # Atualiza a taxa de aprendizado
        optimizer.zero_grad() # Zera os gradientes

     # Retorna a precisão e a perda média por exemplo
    return correct_predictions.double() / n_examples, np.mean(losses)


In [12]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval() # Coloca o modelo em modo de avaliação

    losses = []
    correct_predictions = 0

    with torch.no_grad(): # Desativa o cálculo dos gradientes
        for d in tqdm(data_loader, desc="Evaluating"): # Loop sobre batches de dados
            input_ids = d["input_ids"].to(device) # Move input_ids para o dispositivo
            attention_mask = d["attention_mask"].to(device) # Move attention_mask para o dispositivo
            labels = d["labels"].to(device) # Move labels para o dispositivo

            # Passa os dados pelo modelo sem calcular gradientes
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            loss = loss_fn(outputs.logits, labels) # Calcula a perda
            correct_predictions += torch.sum(torch.argmax(outputs.logits, dim=1) == labels) # Conta predições corretas
            losses.append(loss.item()) # Armazena a perda para análise

    # Retorna a precisão e a perda média por exemplo
    return correct_predictions.double() / n_examples, np.mean(losses)

In [13]:
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0):
        # Inicializa o early stopping com paciência e delta mínimos
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        # Checa se a perda de validação melhorou
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss + self.min_delta:
            self.counter += 1 # Incrementa o contador se a perda não melhorar
            if self.counter >= self.patience:
                self.early_stop = True # Sinaliza o early stopping se o contador atingir a paciência
        else:
            self.best_loss = val_loss
            self.counter = 0 # Reseta o contador se houver melhoria na perda


In [14]:
EPOCHS = 40
early_stopping = EarlyStopping(patience=3, min_delta=0.001) # Configura early stopping

# Loop de treinamento e validação por várias épocas
for epoch in range(EPOCHS):
    start_time = time.time()

    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    # Treina o modelo para uma época e avalia a precisão e perda
    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        device,
        scheduler,
        len(train_df)
    )

    val_acc, val_loss = eval_model(
        model,
        val_data_loader,
        loss_fn,
        device,
        len(val_df)
    )

    end_time = time.time()
    epoch_time = end_time - start_time

    # Exibe as métricas de perda e precisão para treino e validação
    print(f"Perda Treinamento  {train_loss:.4f} precisão {train_acc:.4f}")
    print(f"Perda Validação  {val_loss:.4f} precisão {val_acc:.4f}")
    print(f"Epoch {epoch + 1} demorou {epoch_time // 60:.0f}m {epoch_time % 60:.0f}s")
    print()

    early_stopping(val_loss) # Verifica se deve parar o treinamento antecipadamente

    if early_stopping.early_stop:
        print("Early stopping")
        break

Epoch 1/40
----------


Training:   0%|          | 0/975 [00:00<?, ?it/s]c:\Users\mathe\anaconda3\envs\detector\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Evaluating: 100%|██████████| 209/209 [01:23<00:00,  2.51it/s]


Perda Treinamento  0.1913 precisão 0.9340
Perda Validação  0.1392 precisão 0.9638
Epoch 1 demorou 13m 29s

Epoch 2/40
----------


Evaluating: 100%|██████████| 209/209 [01:26<00:00,  2.41it/s]


Perda Treinamento  0.0569 precisão 0.9859
Perda Validação  0.1118 precisão 0.9770
Epoch 2 demorou 13m 41s

Epoch 3/40
----------


...

Perda Treinamento  0.0173 precisão 0.9958
Perda Validação  0.1169 precisão 0.9806
Epoch 3 demorou 13m 45s

Epoch 4/40
----------


Evaluating: 100%|██████████| 209/209 [01:26<00:00,  2.41it/s]


Perda Treinamento  0.0056 precisão 0.9988
Perda Validação  0.1169 precisão 0.9806
Epoch 4 demorou 13m 46s

Epoch 5/40
----------


Evaluating: 100%|██████████| 209/209 [01:26<00:00,  2.40it/s]

Perda Treinamento  0.0062 precisão 0.9987
Perda Validação  0.1169 precisão 0.9806
Epoch 5 demorou 13m 48s

Early stopping


In [15]:
# Avalia o modelo no conjunto de teste após o término do treinamento
test_acc, _ = eval_model(
    model,
    test_data_loader,
    loss_fn,
    device,
    len(test_df)
)

# Exibe a acurácia final no conjunto de teste
print(f'Teste acurácia: {test_acc.item()}')


Evaluating: 100%|██████████| 209/209 [01:26<00:00,  2.41it/s]

Teste acurácia: 0.9748728686808256


In [16]:
# Salva o modelo
model_save_path = os.path.join(os.curdir, "bert_model.bin")
torch.save(model.state_dict(), model_save_path)
print(f"Modelo salvo em {model_save_path}")

# Salva o tokenizer
tokenizer_save_path = os.path.join(os.curdir, "bert_tokenizer")
bert_tokenizer.save_pretrained(tokenizer_save_path)
print(f"Tokenizer salvo em {tokenizer_save_path}")

Modelo salvo em .\bert_model.bin
Tokenizer salvo em .\bert_tokenizer


In [17]:
def get_predictions(model, data_loader):
    model = model.eval()

    texts = []
    predictions = []
    prediction_probs = []
    real_values = []

    with torch.no_grad():
        for d in data_loader:
            texts.extend(d["input_ids"].tolist())
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, preds = torch.max(outputs.logits, dim=1)

            texts.extend(d["input_ids"].tolist())
            predictions.extend(preds)
            prediction_probs.extend(outputs.logits)
            real_values.extend(labels)

    predictions = torch.stack(predictions).cpu()
    prediction_probs = torch.stack(prediction_probs).cpu()
    real_values = torch.stack(real_values).cpu()
    return predictions, prediction_probs, real_values

# Obtem previsões no conjunto de teste
y_pred, y_pred_probs, y_test = get_predictions(model, test_data_loader)

# Adiciona as previsões ao DataFrame de teste
test_df['predicted_label'] = y_pred
test_df['predicted_label'] = test_df['predicted_label'].apply(lambda x: le.inverse_transform([x])[0])

# Salva o DataFrame de teste
test_save_path = os.path.join(os.curdir, "test_with_predictions.csv")
test_df.to_csv(test_save_path, index=False)
print(f"Dataset de teste salvo em {test_save_path}")

Dataset de teste salvo em .\test_with_predictions.csv
